In [14]:
import nibabel as nib
import os
import matplotlib.pyplot as plt
import numpy as np
import cv2
from einops.layers.torch import Rearrange
import torch
import torchvision
import scipy
import pandas as pd
from sklearn.model_selection import train_test_split

In [24]:
csv_file = r"C:\Users\razva\Master1\Thesis\adni_data.csv"
csv_data = pd.read_csv(csv_file)
csv_data = csv_data.fillna("")
csv_data = csv_data[csv_data["File Path"] != ""]

val_size = int(0.20 * len(csv_data))

train_csv_data, val_csv_data = train_test_split(csv_data, test_size=val_size)
print(len(csv_data))
print(len(train_csv_data))
print(len(val_csv_data))

# train_csv_data.to_csv(r"C:\Users\razva\Master1\Thesis\train_data.csv")
# val_csv_data.to_csv(r"C:\Users\razva\Master1\Thesis\val_data.csv")

train_csv_data

6802
5442
1360


,Unnamed: 0,Image Data ID,Subject,Group,Sex,Age,Visit,Modality,Description,Type,Acq Date,Format,Downloaded,Image Shape,File Path
7309,7309,I225325,941_S_1194,CN,M,89,m48,fMRI,relCBF,Original,3/30/2011,DCM,10/09/2024,"(64, 64, 24)",.\Data\ADNI\941_S_1194\relCBF\2011-03-30_09_50...
5933,5933,I273579,037_S_4410,CN,F,69,v02,fMRI,MoCoSeries,Original,12/19/2011,DCM,10/09/2024,"(64, 64, 24, 105)",.\Data\ADNI\037_S_4410\MoCoSeries\2011-12-19_0...
4352,4352,I532736,116_S_4199,EMCI,M,63,v41,fMRI,MoCoSeries,Original,10/22/2015,DCM,10/08/2024,"(64, 64, 24, 105)",.\Data\ADNI\116_S_4199\MoCoSeries\2015-10-22_1...
1765,1765,I270650,009_S_4359,LMCI,M,77,v02,fMRI,relCBF,Original,12/05/2011,DCM,10/08/2024,"(64, 64, 24)",.\Data\ADNI\009_S_4359\relCBF\2011-12-05_14_26...
3116,3116,I505866,036_S_2380,EMCI,M,72,v31,fMRI,relCBF,Original,8/19/2015,DCM,10/08/2024,"(64, 64, 24)",.\Data\ADNI\036_S_2380\relCBF\2015-08-19_08_04...
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4406,4406,I315439,137_S_4331,EMCI,F,76,v05,fMRI,MoCoSeries,Original,6/20/2012,DCM,10/08/2024,"(64, 64, 24, 105)",.\Data\ADNI\137_S_4331\MoCoSeries\2012-06-20_0...
7811,7811,I1568736,067_S_7063,CN,F,56,sc,fMRI,Axial_MB_rsfMRI__Eyes_Open_,Original,4/13/2022,DCM,10/09/2024,"(88, 88, 64, 976)",.\Data\ADNI\067_S_7063\Axial_MB_rsfMRI__Eyes_O...
2232,2232,I396727,036_S_4899,LMCI,M,83,v11,fMRI,Perfusion_Weighted,Original,10/31/2013,DCM,10/09/2024,"(64, 64, 24)",.\Data\ADNI\036_S_4899\Perfusion_Weighted\2013...
4305,4305,I347729,137_S_4678,EMCI,F,73,v05,fMRI,relCBF,Original,11/26/2012,DCM,10/09/2024,"(64, 64, 24)",.\Data\ADNI\137_S_4678\relCBF\2012-11-26_12_21...


In [43]:
import copy


def normalize_frame(frame):
    """Normalize the frame data to 0-255."""
    frame = cv2.normalize(frame, None, 0, 255, cv2.NORM_MINMAX)
    return frame.astype(np.uint8)

# Load the image
path = r"C:\Users\razva\Master1\Thesis\Data\ADNI\002_S_4213\Axial_MB_rsfMRI__Eyes_Open_\2019-10-03_10_19_44.0\I1236684_nii\I1236684_Axial_MB_rsfMRI_(Eyes_Open)_20191003101944_11.nii.gz"
img = nib.load(path)
data = img.get_fdata()[:, :, :, :100]
# data = np.zeros(shape=[64, 64, 48, 105])
print(data.shape)

if len(data.shape) < 4:
    data = np.expand_dims(data, axis=-1)

print(data.shape)

TARGET_SPACE_DIM = [64, 64, 48]
zoom_factor = (TARGET_SPACE_DIM[0] / data.shape[0], TARGET_SPACE_DIM[1] / data.shape[1], TARGET_SPACE_DIM[2] / data.shape[2], 1)

TARGET_TIME_DIM = 140

print(data.shape)

crt_space_dim = list(data.shape[:3])
crt_time_dim = data.shape[-1]

if crt_time_dim >= TARGET_TIME_DIM:
    data = data[:, :, :, :TARGET_TIME_DIM]
else:
    padded_data = np.zeros(shape=(crt_space_dim + [TARGET_TIME_DIM - crt_time_dim]))
    print(padded_data.shape)
    data = np.concatenate([data, padded_data], axis=3)

print(data.shape)
    
data = scipy.ndimage.zoom(data, zoom_factor)

print(data.shape)

# Extract shape dynamically
x_dim, y_dim, z_dim, time_stamps = data.shape

# Ensure frame size is even for OpenCV compatibility
frame_size = 256
frame_size = frame_size if frame_size % 2 == 0 else frame_size - 1  

num_cols = int(np.ceil(np.sqrt(time_stamps)))
num_rows = int(np.ceil(time_stamps / num_cols))

dest_folder = "movies"
os.makedirs(dest_folder, exist_ok=True)

fourcc = cv2.VideoWriter_fourcc(*'mp4v')

for slice_index in range(x_dim):
    video_path = os.path.join(dest_folder, f"slice_{slice_index}.mp4")
    out = cv2.VideoWriter(video_path, fourcc, 10, (frame_size, frame_size), isColor=True)

    for temp_index in range(time_stamps):
        frame = data[slice_index, :, :, temp_index]
        frame = normalize_frame(frame)

        rot_frame = np.rot90(frame)
        resized_frame = cv2.resize(rot_frame, (frame_size, frame_size))
        
        # Convert grayscale to BGR explicitly
        color_frame = cv2.cvtColor(resized_frame, cv2.COLOR_GRAY2BGR)
        
        out.write(color_frame)  # Ensure frames are written

    out.release()

print("All videos saved successfully.")

(88, 88, 64, 100)
(88, 88, 64, 100)
(88, 88, 64, 100)
(88, 88, 64, 40)
(88, 88, 64, 140)
(64, 64, 48, 140)
All videos saved successfully.


In [44]:
root = r"C:\Users\razva\Master1\Thesis\Data\ADNI"
str = r"C:\Users\razva\Master1\Thesis\Data\ADNI\002_S_4213\Axial_MB_rsfMRI__Eyes_Open_\2019-10-03_10_19_44.0\I1236684_nii"
str.replace(root, "")

'\\002_S_4213\\Axial_MB_rsfMRI__Eyes_Open_\\2019-10-03_10_19_44.0\\I1236684_nii'